In [11]:
Packages <- c("dplyr",  "nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer", "viridis")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
#source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_grid_search.R")
source("~/oceanography/scripts/LL_biophys.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R")
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained
source("~/oceanography/scripts/format_genetic_parentage_matrix.R")
source("~/oceanography/scripts/format_genetic_kernel_parentage_matrix.R")
source("~/oceanography/scripts/format_biophys_normalized_matrix.R")
source("~/oceanography/scripts/format_biophys_parentage_matrix.R")
set.seed(7)
#source("~/oceanography/scripts/PredictedProportions.R")

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
kernel2012_14 <- fread(file="~/oceanography/empirical_data/genetics/GenKernelsForROMSComp2012-14.csv")

#read in the necessary input data for likelihood functions
load(file= "~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_InputSurveyData.Rdata")
load(file= "~/oceanography/script_output/ROMSDataTables/2021-11-04_SeededParticleInfo.Rdata")
load(file="~/oceanography/script_output/SurveyData/for_likelihood_functions/2021-11-04_AddDestTables.Rdata")

In [2]:
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
#head(AllRecruitsGen)
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")
#head(MonsoonRecSampPar)

#make one data table to rule them all....

AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
    offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]

genetic_parentage_matrices <- suppressMessages(format_genetic_parentage_matrix(AllRecruitsGenWithMonsoon, SurveyData))
#fwrite(genetic_parentage_matrices$GenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2012, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2013, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMat2014, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_parentage_matrices$GenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_ParentageMatrixSWM2012-14ForROMSComp.csv")


genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoon, SurveyData))

#fwrite(genetic_kernel_matrices$KernelGenMat2012_4, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2012, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2012ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2013, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2013ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMat2014, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrix2014ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatNEM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixNEM2012-14ForROMSComp.csv")
#fwrite(genetic_kernel_matrices$KernelGenMatSWM, file="~/oceanography/script_output/SurveyData/20210701_KernelParentageMatrixSWM2012-14ForROMSComp.csv")


__read in the biophysical data and format for likelihood functions__

In [177]:
SimConn <- fread(file="~/oceanography/script_output/ROMSDataTables/SimConnectivityTableCompleteMetaLongForm08DayPLD.csv")[year %in% c(2012, 2013, 2014) & dest != "CAI"][#filter out CAI as a destination for now, not very well spatially defined
   ,`:=`(daily_particles_rec=as.numeric(daily_particles_rec),daily_particles_released=as.numeric(daily_particles_released))][
    source == "Other" | source == "CAI", source:="unknown"][ #aggregate the CAI and other sites as "unknown for likelihood functions"
    , daily_particles_rec := sum(daily_particles_rec), by=c("date", "source", "dest")][
   , daily_particles_released :=sum(as.numeric(daily_particles_released)), by=c("date", "source")]
total_release_days

[1] 687

In [194]:
AddDestSim <- rbind(rbindlist(list(unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2012],
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2013], 
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2014]))[, monsoon := "NEM"],
    rbindlist(list(unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2012],
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2013], 
        unique(SimConn[, .(source, dest)], by=c("source", "dest"))[, year := 2014]))[, monsoon := "SWM"])

In [195]:
##add in sites that aren't represented in every year by the genetics for the biophysical PARENTAGE matrix used to fit the biophysical kernel
AddDestGenAnnual <- rbindlist(list(unique(cbind(SurveyData[year==2012 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2012 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2012],  #what destinations were sampled, for use with unassigned table
unique(cbind(SurveyData[year==2013 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2013 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2013],
unique(cbind(SurveyData[year==2014 & prop_anem_samp >0 ][, .(offs_site=site)], 
                  SurveyData[year==2014 & prop_anem_samp >0 ][ , .(par_site=site)]), by=c("offs_site", "par_site"))[, year := 2014]))

#AddDestGen <- cbind(PropSamp[prop_anem_samp > 0 & year==2014, .(site)][, .(offs_site=site)], PropSamp[prop_anem_samp > 0 & year==2014, .(site)][, .(par_site=site)])

AddDestGenMonsoon <- rbind(cbind(AddDestGenAnnual, data.table(monsoon = rep("NEM",nrow(AddDestGenAnnual)))), cbind(AddDestGenAnnual, data.table(monsoon = rep("SWM",nrow(AddDestGenAnnual)))))


In [197]:
AddDestGenAnnual

offs_site,par_site,year
Cabatoan,Cabatoan,2012
Magbangon,Magbangon,2012
Palanas,Palanas,2012
Poroc Rose,Poroc Rose,2012
Poroc San Flower,Poroc San Flower,2012
San Agustin,San Agustin,2012
Sitio Lonas,Sitio Lonas,2012
Visca,Visca,2012
Wangag,Wangag,2012
Cabatoan,Cabatoan,2013


In [81]:

biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConn, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestAnnualSim=AddDestAnnualSim, AddDestMonsoonSim=AddDestMonsoonSim)

#fwrite(biophys_norm_matrices$FullBiophysMatNorm, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2012ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2013ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$AnnualBiophysMatNorm2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrix2014ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormNEM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixNEM2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_norm_matrices$MonsoonBiophysMatNormSWM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysNormConnMatrixSWM2012-14ForROMSComp15DayPLD.csv")

biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConn, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestGenAnnual=AddDestGenAnnual)
#fwrite(biophys_parentage_matrices$FullBiophysMat, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2012, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2012ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2013, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2013ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$AnnualBiophysMat2014, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrix2014ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatNEM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixNEM2012-14ForROMSComp15DayPLD.csv")
#fwrite(biophys_parentage_matrices$MonsoonBiophysMatSWM, file="~/oceanography/script_output/ROMSDataTables/20210917_BioPhysParentageMatrixSWM2012-14ForROMSComp15DayPLD.csv")

In [95]:
head(SimConn)

particle_id,source,dest,year,monsoon,date,num_route_parentage_matches,num_release_days_seasonal,num_release_days_annual,dest_num_females,...,source_num_rec_sampled_annual,source_prop_anem_samp,sim_month,sim_day,sim_year,daily_particles_rec,daily_particles_released,dist_km,bearing,direction
P54642131,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642132,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642133,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642134,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642135,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA
P54642136,Other,Palanas,2012,SWM,2011-10-01,3,31,244,49,...,NA,NA,10,10,2011,69,4497728,NA,NA,NA


In [84]:
sum(biophys_parentage_matrices$AnnualBiophysMat2012)+sum(biophys_parentage_matrices$AnnualBiophysMat2013)+sum(biophys_parentage_matrices$AnnualBiophysMat2014)
sum(biophys_parentage_matrices$FullBiophysMat)

[1] 643203

[1] 867251

In [131]:
biophys_parentage_matrices$AnnualBiophysMat2012

Cabatoan,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Lonas,Visca,Wangag
454,429,66,13,22,87,87,0,126
400,175,177,6,15,71,71,0,215
209,143,42,0,0,0,0,0,239
63,7,6,273,506,811,2011,99,13
17,16,7,594,788,1768,3255,163,16
114,86,9,1006,993,2159,4072,61,15
207,163,28,6,18,278,8805,9,18
33,28,3,388,736,1615,2608,115,1
412,593,221,4,0,4,7,0,447
7705,7944,7199,1714,2276,4265,5719,2807,7440


In [130]:
biophys_parentage_matrices$FullBiophysMat

Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Tamakin Dacot,Visca,Wangag
1002,1132,1769,235,48,922,787,204,71,136,381,940,375,55,1,283
766,2242,2922,202,12,245,763,129,78,146,539,902,662,2,0,126
774,731,2096,186,8,264,522,48,56,197,700,761,667,4,2,89
447,909,1762,1582,39,3832,436,88,184,286,838,530,721,8,2,95
21,1,11,127,3475,40,14,4,28,36,91,62,28,1317,76,8
1076,2272,3675,1973,479,4098,753,222,674,1125,1847,1888,2154,267,77,351
759,1862,2692,177,46,805,462,372,41,83,532,1641,269,23,13,542
665,1481,803,2,0,93,799,631,9,1,38,3136,1,0,0,560
146,325,1016,1036,185,735,38,81,757,1134,2265,147,5030,105,678,51
179,226,818,1688,181,808,80,80,1180,1579,5073,228,8516,103,463,94


In [178]:
sum(AllYearsRec$total_particles_rec)
sum(AnnualRec$annual_particles_rec)
nrow(SimConn)

[1] 955263

[1] 955263

[1] 955263

In [104]:
sum(AllYearsNormConn[source!="unknown",total_particles_rec])
sum(AnnualNormConn[source!="unknown",annual_particles_rec])

[1] 463099

[1] 463099

In [129]:
dim(biophys_par_data2012_4$Assignments)
dim(biophys_par_data2012$Assignments)

(biophys_norm_matrices$FullBiophysMatNorm)
dim(genetic_parentage_matrices$GenMat2012)

[1] 17 16

[1] 10  9

Cabatoan,Caridad Cemetery,Caridad Proper,Elementary School,Gabas,Haina,Hicgop South,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Baybayon,Sitio Lonas,Sitio Tugas,Tamakin Dacot,Visca,Wangag
1.470278e-03,1.661032e-03,2.595729e-03,3.448256e-04,0.000000e+00,7.043246e-05,1.352890e-03,1.154799e-03,2.993379e-04,1.041813e-04,1.995586e-04,5.590576e-04,1.379302e-03,5.502536e-04,4.900925e-04,8.070386e-05,1.467343e-06,4.152580e-04
1.123985e-03,3.289783e-03,4.287576e-03,2.964032e-04,1.173874e-05,1.760811e-05,3.594990e-04,1.119583e-03,1.892872e-04,1.144527e-04,2.142321e-04,7.908978e-04,1.323543e-03,9.713809e-04,4.724844e-04,2.934686e-06,0.000000e+00,1.848852e-04
1.135723e-03,1.072628e-03,3.075551e-03,2.729258e-04,0.000000e+00,1.173874e-05,3.873785e-04,7.659530e-04,7.043246e-05,8.217120e-05,2.890665e-04,1.027140e-03,1.116648e-03,9.787177e-04,4.548763e-04,5.869371e-06,2.934686e-06,1.305935e-04
6.559022e-04,1.333815e-03,2.585458e-03,2.321336e-03,1.467343e-06,5.722637e-05,5.622858e-03,6.397615e-04,1.291262e-04,2.699911e-04,4.196600e-04,1.229633e-03,7.776917e-04,1.057954e-03,2.701378e-03,1.173874e-05,2.934686e-06,1.393976e-04
1.716791e-04,1.379302e-04,5.194394e-04,2.677901e-03,2.502993e-02,2.377095e-04,1.760811e-03,1.511363e-04,4.548763e-05,1.194417e-03,1.810701e-03,8.252336e-03,2.435789e-04,1.558465e-02,2.299326e-03,1.804832e-04,1.260447e-02,3.228154e-05
1.540710e-05,7.336714e-07,8.070386e-06,9.317627e-05,1.467343e-05,2.549508e-03,2.934686e-05,1.027140e-05,2.934686e-06,2.054280e-05,2.641217e-05,6.676410e-05,4.548763e-05,2.054280e-05,4.108560e-05,9.662452e-04,5.575903e-05,5.869371e-06
7.894304e-04,1.666901e-03,2.696242e-03,1.447534e-03,3.668357e-05,3.514286e-04,3.006585e-03,5.524546e-04,1.628751e-04,4.944945e-04,8.253803e-04,1.355091e-03,1.385172e-03,1.580328e-03,1.258246e-03,1.958903e-04,5.649270e-05,2.575187e-04
1.113713e-03,2.732192e-03,3.950087e-03,2.597197e-04,0.000000e+00,6.749777e-05,1.181211e-03,6.779124e-04,5.458515e-04,6.016106e-05,1.217895e-04,7.806264e-04,2.407910e-03,3.947152e-04,3.448256e-04,3.374888e-05,1.907546e-05,7.952998e-04
9.757830e-04,2.173135e-03,1.178276e-03,2.934686e-06,0.000000e+00,0.000000e+00,1.364629e-04,1.172407e-03,9.258933e-04,1.320609e-05,1.467343e-06,5.575903e-05,4.601587e-03,1.467343e-06,7.483448e-05,0.000000e+00,0.000000e+00,8.217120e-04
2.142321e-04,4.768864e-04,1.490820e-03,1.520167e-03,1.614077e-05,2.714584e-04,1.078497e-03,5.575903e-05,1.188548e-04,1.110779e-03,1.663967e-03,3.323531e-03,2.156994e-04,7.380734e-03,1.570057e-03,1.540710e-04,9.948584e-04,7.483448e-05


[1] 19 18

In [118]:
#==unique(AddDestGenAnnual$par_site)
SurveyData[site %!in% unique(AddDestGenAnnual$offs_site)]
SurveyData

year,site,num_females,n_offs_gen,prop_anem_samp
2012,Gabas,NA,0,0.0000000
2012,Sitio Tugas,NA,0,0.0000000
2013,Gabas,NA,0,0.0000000
2013,Sitio Tugas,4,0,0.0000000
2014,Gabas,NA,0,0.0000000
2014,Sitio Tugas,NA,0,0.0000000
2015,Gabas,8,1,0.1111111
2015,Sitio Tugas,NA,0,0.0000000
2016,Gabas,12,1,0.1777778
2016,Sitio Tugas,NA,0,0.0000000


year,site,num_females,n_offs_gen,prop_anem_samp
2012,Cabatoan,8,4,0.42307692
2012,Caridad Cemetery,NA,0,0.00000000
2012,Caridad Proper,NA,0,0.00000000
2012,Elementary School,NA,0,0.00000000
2012,Gabas,NA,0,0.00000000
2012,Haina,NA,0,0.00000000
2012,Hicgop South,NA,0,0.00000000
2012,Magbangon,101,10,0.45938375
2012,Palanas,49,13,0.28985507
2012,Poroc Rose,9,9,1.00000000


In [108]:
sum(dcast(AllYearsNormConn[source != "unknown", .(source, dest, total_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "total_particles_rec")[, 
        -"source"], na.rm=T)

[1] 463099

In [147]:
head(AllYearsRec)

source,dest,total_particles_rec
Other,Palanas,20264
Other,Wangag,18382
Other,Magbangon,18567
Other,Cabatoan,18851
Other,Caridad Cemetery,21764
Other,Caridad Proper,24084


In [ ]:
[source == "Other" | 
        source == "CAI", source:="unknown"][, total_particles_rec :=
        sum(total_particles_rec)), by = c("source", "dest")], 
        by = c("source", "dest"))

In [179]:
#double checked, this is correct***
AllYearsRec <- SimConn[, .(total_particles_rec = .N), by = c("source", 
        "dest")]
AllYearsRelease <- unique(SimConn[, .(total_particles_released = as.numeric(daily_particles_released) * 
        as.numeric(total_release_days)), by = c("source")], by = "source")
AllYearsNormConn <- AllYearsRelease[AllYearsRec, on = "source"][ 
        , source_norm_rec:= total_particles_rec/total_particles_released]
FullBiophysMat <- as.matrix(rbind(dcast(AllYearsNormConn[source != "unknown", .(source, dest, total_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "total_particles_rec")[, 
        -"source"], dcast(AllYearsNormConn[source == "unknown", .(source, dest, total_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "total_particles_rec")[, 
        -"source"]))
    FullBiophysMat[is.na(FullBiophysMat)] <- 0
sum(FullBiophysMat)

In [193]:
AddDestAnnualSim

source,dest,year
Other,Palanas,2012
Other,Wangag,2012
Other,Magbangon,2012
Other,Cabatoan,2012
Other,Caridad Cemetery,2012
Other,Caridad Proper,2012
Other,Hicgop South,2012
Other,Sitio Tugas,2012
Other,Elementary School,2012
Other,Sitio Lonas,2012


In [ ]:
AnnualRec <- SimConn[, .(annual_particles_rec = .N), by = c("source", 
        "dest", "year")]
    AnnualRelease <- unique(SimConn[, .(annual_particles_released = as.numeric(daily_particles_released) * 
        as.numeric(num_release_days_annual)), by = c("source", 
        "year")], by = c("source", "year"))
    AnnualRecInt <- rbind(AddDestAnnualSim[!AnnualRec, on = .(source, 
        dest, year)][, annual_particles_rec := 0)], AnnualRec)
    AnnualReleaseInt <- rbind(AddDestAnnualSim[!AnnualRelease, 
        on = .(source, year)][, annual_particles_released :=  0)][, -"dest"], AnnualRelease)
    AnnualRec <- unique(AnnualRecInt[source == "Other" | source == 
        "CAI", `:=`(source, "unknown")][, `:=`(annual_particles_rec, 
        sum(annual_particles_rec)), by = c("source", "dest", 
        "year")], by = c("source", "dest", "year"))
    AnnualRelease <- unique(AnnualReleaseInt[source == "Other" | 
        source == "CAI", `:=`(source, "unknown")][, `:=`(annual_particles_released, 
        sum(annual_particles_released)), by = c("source", "year")], 
        by = c("source", "year"))
    AnnualNormConn <- AnnualRelease[AnnualRec, on = c("source", 
        "year")][, `:=`(source_norm_rec, annual_particles_rec/annual_particles_released)]


In [87]:
    AllYearsRec <- SimConn[, .(total_particles_rec = .N), by = c("source", 
        "dest")]
    AllYearsRelease <- unique(SimConn[, .(total_particles_released = as.numeric(daily_particles_released) * 
        as.numeric(total_release_days)), by = c("source")], by = "source")
    AllYearsRecInt <- rbind(AddDestAllYearsSim[!AllYearsRec, 
        on = .(source, dest)][, `:=`(total_particles_rec, 0)], 
        AllYearsRec)
    AllYearsReleaseInt <- rbind(AddDestAllYearsSim[!AllYearsRelease, 
        on = .(source)][, `:=`(total_particles_released, 0)][, 
        -"dest"], AllYearsRelease)
    AllYearsRec <- unique(AllYearsRecInt[source == "Other" | 
        source == "CAI", `:=`(source, "unknown")][, `:=`(total_particles_rec, 
        sum(total_particles_rec)), by = c("source", "dest")], 
        by = c("source", "dest"))
    AllYearsRelease <- unique(AllYearsReleaseInt[source == "Other" | 
        source == "CAI", `:=`(source, "unknown")][, `:=`(total_particles_released, 
        sum(total_particles_released)), by = "source"], by = "source")
    AllYearsNormConn <- AllYearsRelease[AllYearsRec, on = "source"][, 
        `:=`(source_norm_rec= total_particles_rec/total_particles_released)]
    FullBiophysMat <- as.matrix(rbind(dcast(AllYearsNormConn[source %in% 
        AddDestGenAnnual[, par_site] & dest %in% AddDestGenAnnual[, 
        offs_site]][source != "unknown", .(source, dest, total_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "total_particles_rec")[, 
        -"source"], dcast(AllYearsNormConn[dest %in% AddDestGenAnnual[, 
        offs_site]][source == "unknown", .(source, dest, total_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "total_particles_rec")[, 
        -"source"]))
    FullBiophysMat[is.na(FullBiophysMat)] <- 0
    AnnualRec <- SimConn[, .(annual_particles_rec = .N), by = c("source", 
        "dest", "year")]
    AnnualRelease <- unique(SimConn[, .(annual_particles_released = as.numeric(daily_particles_released) * 
        as.numeric(num_release_days_annual)), by = c("source", 
        "year")], by = c("source", "year"))
    AnnualRecInt <- rbind(AddDestAnnualSim[!AnnualRec, on = .(source, 
        dest, year)][, `:=`(annual_particles_rec, 0)], AnnualRec)
    AnnualReleaseInt <- rbind(AddDestAnnualSim[!AnnualRelease, 
        on = .(source, year)][, `:=`(annual_particles_released, 
        0)][, -"dest"], AnnualRelease)
    AnnualRec <- unique(AnnualRecInt[source == "Other" | source == 
        "CAI", `:=`(source, "unknown")][, `:=`(annual_particles_rec, 
        sum(annual_particles_rec)), by = c("source", "dest", 
        "year")], by = c("source", "dest", "year"))
    AnnualRelease <- unique(AnnualReleaseInt[source == "Other" | 
        source == "CAI", `:=`(source, "unknown")][, `:=`(annual_particles_released, 
        sum(annual_particles_released)), by = c("source", "year")], 
        by = c("source", "year"))
    AnnualNormConn <- AnnualRelease[AnnualRec, on = c("source", 
        "year")][, `:=`(source_norm_rec, annual_particles_rec/annual_particles_released)]
    AnnualBiophysMat2012 <- as.matrix(rbind(dcast(AnnualNormConn[year == 
        2012][source %in% AddDestGenAnnual[year == 2012, par_site] & 
        dest %in% AddDestGenAnnual[year == 2012, offs_site]][source != 
        "unknown", .(source, dest, annual_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "annual_particles_rec")[, 
        -"source"], dcast(AnnualNormConn[year == 2012][dest %in% 
        AddDestGenAnnual[year == 2012, offs_site]][source == 
        "unknown", .(source, dest, annual_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "annual_particles_rec")[, 
        -"source"]))
    AnnualBiophysMat2012[is.na(AnnualBiophysMat2012)] <- 0
    AnnualBiophysMat2013 <- as.matrix(rbind(dcast(AnnualNormConn[year == 
        2013][source %in% AddDestGenAnnual[year == 2013, par_site] & 
        dest %in% AddDestGenAnnual[year == 2013, offs_site]][source != 
        "unknown", .(source, dest, annual_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "annual_particles_rec")[, 
        -"source"], dcast(AnnualNormConn[year == 2013][dest %in% 
        AddDestGenAnnual[year == 2013, offs_site]][source == 
        "unknown", .(source, dest, annual_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "annual_particles_rec")[, 
        -"source"]))
    AnnualBiophysMat2013[is.na(AnnualBiophysMat2013)] <- 0
    AnnualBiophysMat2014 <- as.matrix(rbind(dcast(AnnualNormConn[year == 
        2014][source %in% AddDestGenAnnual[year == 2014, par_site] & 
        dest %in% AddDestGenAnnual[year == 2014, offs_site]][source != 
        "unknown", .(source, dest, annual_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "annual_particles_rec")[, 
        -"source"], dcast(AnnualNormConn[year == 2014][dest %in% 
        AddDestGenAnnual[year == 2014, offs_site]][source == 
        "unknown", .(source, dest, annual_particles_rec)][order(source, 
        dest)], source ~ dest, value.var = "annual_particles_rec")[, 
        -"source"]))
    AnnualBiophysMat2014[is.na(AnnualBiophysMat2014)] <- 0

In [88]:
AllYearsNormConn

source,total_particles_released,dest,total_particles_rec,source_norm_rec
unknown,3259633632,Palanas,20296,6.226467e-06
unknown,3259633632,Wangag,18392,5.642352e-06
unknown,3259633632,Magbangon,18610,5.709231e-06
unknown,3259633632,Cabatoan,18877,5.791142e-06
unknown,3259633632,Caridad Cemetery,21764,6.676824e-06
unknown,3259633632,Caridad Proper,24086,7.389174e-06
unknown,3259633632,Hicgop South,33600,1.030791e-05
unknown,3259633632,Sitio Tugas,12731,3.905654e-06
unknown,3259633632,Elementary School,21110,6.476188e-06
unknown,3259633632,Sitio Lonas,19517,5.987483e-06


In [8]:
#12/9/2021- I have a spare hour while something else runs and I'm wondering if including some sampling of particles like our data would make for a helpful likelihood comparison
TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]
#head(TotalOffsBySiteYear)
SimConnSubSamp <- left_join(SimConn,TotalOffsBySiteYear, by=c("year", dest="site"))

SimConnInt2 <- SimConnSubSamp[!is.na(num_gen), .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "dest")] 
nrow(SimConnInt2)#should be 394



[1] 394

In [13]:
 AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
        offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 #    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site

TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]

AllRecruitsGenWithMonsoon2 <- left_join(AllRecruitsGenWithMonsoon, TotalOffsBySiteYear, by=c("year", "site"))


In [41]:
sum(par_data2012$Assignments[1:9,])
par_data2012$Assignments[1:9,]

[1] 5

Cabatoan,Magbangon,Palanas,Poroc Rose,Poroc San Flower,San Agustin,Sitio Lonas,Visca,Wangag
0,0,0,0,0,0,0,0,0
0,0,0,0,2,0,0,0,0
0,0,0,0,0,0,0,0,1
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,0,0,0,0,0
0,0,0,0,2,0,0,0,0


In [51]:
sum(par_data2012$Assignments[1:nrow(par_data2012$Assignments)-1,])

[1] 3

In [31]:
GenKernelsInt
SimKernelsInt

data_source,time_scale,time_id,k,theta,num_matches,mdd,med,dist90,log_like,iteration
genetics,annual,2012,-3.1458301,3.1481900,1,11.65870,10.6105,22.7102,-25.63257,10
genetics,annual,2013,0.1884694,0.4517980,1,7.84544,3.3907,19.8874,-100.42956,10
genetics,annual,2014,0.4468176,0.3716558,11,18.06205,6.4884,45.7550,-54.47936,10
genetics,total_years,2012-4,-0.4616947,0.4917302,36,10.21656,4.7342,25.7905,-188.69090,10
genetics,monsoonal,NEM,1.3750318,0.3206142,4,20.44832,6.2232,51.1506,-20.27176,10
genetics,monsoonal,SWM,-3.3969615,5.0000000,11,14.43384,13.7616,27.0159,-68.39587,10


data_source,time_scale,time_id,k,theta,num_matches,mdd,med,dist90,log_like,iteration
simulation,annual,2012,-2.1641768,5.0000000,38689,4.207172,4.0112,7.8746,-1.485385e+05,10
simulation,annual,2013,-0.4685478,0.6310938,120977,4.211004,2.3232,10.3828,-5.484369e+05,10
simulation,annual,2014,0.2723692,0.5290741,130789,3.642838,1.7847,9.1469,-5.294751e+05,10
simulation,total_years,2012-4,-3.0000000,1.0000000,391474,20.085537,13.9222,46.2487,1.600000e+13,10
simulation,monsoonal,NEM,-0.2354231,0.6091191,375115,3.713663,2.0037,9.1949,-1.567006e+06,10
simulation,monsoonal,SWM,0.6353253,0.4132845,16359,7.954818,3.1739,20.1979,-9.034707e+04,10


In [79]:
sum(biophys_par_data2012_4$Assignments[1:16,])

[1] 392928

In [59]:
Formal

data_source,time_scale,time_id,k,theta,num_matches,mdd,med,dist90,log_like,iteration
simulation,annual,2012,-2.1634553,5.0000000,38716,4.204138,4.0083,7.8689,-1.483071e+05,10
simulation,annual,2013,-0.4730160,0.6324905,121623,4.202406,2.3216,10.3588,-5.503447e+05,10
simulation,annual,2014,0.2863413,0.5271752,130609,3.642430,1.7798,9.1486,-5.282207e+05,10
simulation,total_years,2012-4,-3.0000000,1.0000000,392928,20.085537,13.9222,46.2487,1.600000e+13,10
simulation,monsoonal,NEM,-0.2246377,0.6075131,376515,3.704210,1.9953,9.1742,-1.569939e+06,10
simulation,monsoonal,SWM,0.6475479,0.4126315,16413,7.927018,3.1581,20.1274,-9.044257e+04,10


In [50]:
AllRecruitsGenWithMonsoonInt[year==2012& matched_offs=="Y"]

year,site,fish_indiv,size,color,sex,gen_id,sample_id,date,anem_obs_time,...,gps,dive_type,input,par_fish_indiv,offs_site,par_site,year_match,matched_offs,monsoon,num_gen
2012,Wangag,762,5.1,Y,J,762,APCL12_256,2012-05-14,09:45:00,...,1,E,offspring,236,Wangag,Palanas,2012,Y,SWM,16
2012,Wangag,762,5.1,Y,J,762,APCL12_256,2012-05-14,09:45:00,...,1,E,offspring,236,Wangag,Palanas,2012,Y,SWM,16
2012,Poroc San Flower,224,5.7,W,J,224,APCL12_205,2012-05-11,12:07:00,...,1,E,offspring,196,Poroc San Flower,Magbangon,2012,Y,SWM,9


In [52]:
class(sum(biophys_par_data2012$Assignments[1:nrow(biophys_par_data2012$Assignments)-1,]))

[1] "numeric"

# Get the best-fit kernels and profile likelihood confidence intervals in fit_data.ipynb


In [53]:
sampled_reefs_vec <- as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]) #***I think these index values need to match dimensions of parentage matrix- filter out sand flats and other unsampled sites***
pop_size_vec <- as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]) #vector of pop sizes for all reefs (a). This term is also used in parentage kernel fitting, but reef sizes are substituted as a proxy for pop size. This is should be bootstrapped to account for uncertainty.
prop_samp_vec <- as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)])#vector of proportion of habitat sampled for all reefs in time period t- I think that I should be coming up with a different value for this, but for now this will help me code the function

#bootstrap the particle AND parentage data and re-fit kernels and perform bode likelihood function for dispersal event matching
num_samples_sim <- nrow(SimConn) #because each row in SimConn is a particle the number of rows is N samples per iteration for bootstrapping

#make table to hold results
BootKernels <- data.table(data_source=character(), time_scale=character(), time_id=character(), k=numeric(), theta=numeric(),num_matches=numeric(), mdd=numeric(), med=numeric(), dist90=numeric(), log_like=numeric(), iteration=numeric())

BootLikelihood <- data.table(comparison=character(), time_scale=character(), log_like=numeric(), iteration=numeric())

pb <- txtProgressBar(min = 0, max = 1000, style = 3)
AllRecruitsGen <- fread("~/parentage/kernel_fitting/1340_loci/final_results/tables/AllFishObsWithPar.csv")
MonsoonRecSampPar <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/RecruitsByMonsoon2012-14ForROMSComp.csv")


 AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
        offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 #    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site

TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]

AllRecruitsGenWithMonsoon2 <- left_join(AllRecruitsGenWithMonsoon, TotalOffsBySiteYear, by=c("year", "site"))

for(i in 1:10){
    
    #resample dispersal data, both genetics and simulation
    SimConnInt2 <- SimConn[, .SD[sample(.N, num_samples_sim, replace=TRUE)]] #sample the particle data
    #test case subsampling the particle data
    #SimConnInt2 <- SimConnSubSamp[!is.na(num_gen), .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "dest")] 

#make one data table to rule them all....

 #   AllRecruitsGenWithMonsoon <- left_join(AllRecruitsGen, MonsoonRecSampPar[, .(fish_indiv, year, monsoon=season)], by=c("fish_indiv", "year"))[
 #       offs_site %like% "Magbangon", offs_site := "Magbangon"][par_site %like% "Magbangon", par_site := "Magbangon"][site %like% "Magbangon", site := "Magbangon"][year %in% c(2012, 2013, 2014)]    
 ##    , .SD[sample(.N, 394, replace=TRUE)]] #resample the recruits with replacement not by the same number at each site
 #  
    #TotalOffsBySiteYear <- AllRecruitsGenWithMonsoon[, .(num_gen=.N), by=c("year", "site")]
    
    #test case redoing with subsampling by site and year instead of total recruits sampled
    AllRecruitsGenWithMonsoonInt <- AllRecruitsGenWithMonsoon2[, .SD[sample(.N, num_gen, replace=TRUE)], by=c("year", "site")] #resample the recruits with replacement
    

    
        #format the matrices for kernel fitting and likelihood function
        genetic_parentage_matrices <- suppressMessages(format_genetic_parentage_matrix(AllRecruitsGenWithMonsoon=AllRecruitsGenWithMonsoonInt, SurveyData))
        genetic_kernel_matrices <- suppressMessages(format_genetic_kernel_parentage_matrix(AllRecruitsGenWithMonsoon=AllRecruitsGenWithMonsoonInt, SurveyData))
        biophys_norm_matrices <- format_biophys_normalized_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestAnnualSim=AddDestAnnualSim, AddDestMonsoonSim=AddDestMonsoonSim)
        biophys_parentage_matrices <- format_biophys_parentage_matrix(SimConn=SimConnInt2, total_release_days=687, AddDestAllYearsSim=AddDestAllYearsSim, AddDestGenAnnual=AddDestGenAnnual)



##fit the kernels
biophys_par_data2012 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2012), ncol=1, 1))
Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012, control=list(maxit=500)))

biophys_par_data2013 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2013), ncol=1, 1))
Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2013, control=list(maxit=500)))

biophys_par_data2014 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$AnnualBiophysMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014), ncol=1, 1))
Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2014, control=list(maxit=500)))

biophys_par_data2012_4 <- list(Distances=Distances, Assignments=biophys_parentage_matrices$FullBiophysMat, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$AnnualBiophysMat2014)))
Sim2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_data2012_4, control=list(maxit=500)))

biophys_par_dataNEM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatNEM), ncol=1, 1))
SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataNEM, control=list(maxit=500)))

biophys_par_dataSWM <- list(Distances=Distances, Assignments=biophys_parentage_matrices$MonsoonBiophysMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=matrix(nrow=ncol(biophys_parentage_matrices$MonsoonBiophysMatSWM), ncol=1, 1))
SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=biophys_par_dataSWM, control=list(maxit=500)))

#genetic data with same reef metadata structure as model

par_data2012 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2012 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2012 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012, control=list(maxit=500)))

par_data2013 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2013, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2013 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2013 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2013, control=list(maxit=500)))

par_data2014 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2014, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2014, control=list(maxit=500)))

par_data2012_4 <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMat2012_4, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014 & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014 & prop_anem_samp >0, .(prop_anem_samp)]))
Gen2012_4Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_data2012_4, control=list(maxit=500)))

par_dataNEM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatNEM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataNEM, control=list(maxit=500)))

par_dataSWM <- list(Distances=Distances, Assignments=genetic_kernel_matrices$KernelGenMatSWM, Sampled_reefs=t(as.matrix(SiteIndex[site %in% SurveyData[year==2014  & prop_anem_samp >0 , site], .(index)])), 
                  Reef_sizes=reef_sizes, Adult_sample_proportions=as.matrix(SurveyData[year==2014   & prop_anem_samp >0, .(prop_anem_samp)]))
GenSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 5), method="L-BFGS-B", data=par_dataSWM, control=list(maxit=500)))


#make a summary table for all time frames of simulation data
SimKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 10))
setDT(SimKernelsInt)
setnames(SimKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta","num_matches", "mdd", "med", "dist90", "log_like"))


BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[1] <- i
SimKernelsInt$data_source[1] <- "simulation"
SimKernelsInt$time_scale[1] <- "annual"
SimKernelsInt$time_id[1] <- "2012"
SimKernelsInt$k[1] <-  BestK2012
SimKernelsInt$theta[1] <- BestTheta2012
SimKernelsInt$num_matches[1] <-sum(biophys_par_data2012$Assignments[1:nrow(biophys_par_data2012$Assignments)-1,])
SimKernelsInt$mdd[1] <- MDD2012
SimKernelsInt$med[1] <- Med2012
SimKernelsInt$dist90[1] <- Dist90_2012
SimKernelsInt$log_like[1] <- logLik(Sim2012Fit)[1]

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[2] <- i
SimKernelsInt$data_source[2] <- "simulation"
SimKernelsInt$time_scale[2] <- "annual"
SimKernelsInt$time_id[2] <- "2013"
SimKernelsInt$k[2] <-  BestK2013
SimKernelsInt$theta[2] <- BestTheta2013
SimKernelsInt$num_matches[2] <-sum(biophys_par_data2013$Assignments[1:nrow(biophys_par_data2013$Assignments)-1,])
SimKernelsInt$mdd[2] <- MDD2013
SimKernelsInt$med[2] <- Med2013
SimKernelsInt$dist90[2] <- Dist90_2013
SimKernelsInt$log_like[2] <- logLik(Sim2013Fit)[1]


BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[3] <- i
SimKernelsInt$data_source[3] <- "simulation"
SimKernelsInt$time_scale[3] <- "annual"
SimKernelsInt$time_id[3] <- "2014"
SimKernelsInt$k[3] <-  BestK2014
SimKernelsInt$theta[3] <- BestTheta2014
SimKernelsInt$num_matches[3] <-sum(biophys_par_data2014$Assignments[1:nrow(biophys_par_data2014$Assignments)-1,])
SimKernelsInt$mdd[3] <- MDD2014
SimKernelsInt$med[3] <- Med2014
SimKernelsInt$dist90[3] <- Dist90_2014
SimKernelsInt$log_like[3] <- logLik(Sim2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Sim2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Sim2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)
    
SimKernelsInt$iteration[4] <- i
SimKernelsInt$data_source[4] <- "simulation"
SimKernelsInt$time_scale[4] <- "total_years"
SimKernelsInt$time_id[4] <- "2012-4"
SimKernelsInt$k[4] <-  BestK2012_4
SimKernelsInt$theta[4] <- BestTheta2012_4
SimKernelsInt$num_matches[4] <-sum(biophys_par_data2012_4$Assignments[1:nrow(biophys_par_data2012_4$Assignments)-1,])
SimKernelsInt$mdd[4] <- MDD2012_4
SimKernelsInt$med[4] <- Med2012_4
SimKernelsInt$dist90[4] <- Dist90_2012_4
SimKernelsInt$log_like[4] <- logLik(Sim2012_4Fit)[1]

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[5] <- i
SimKernelsInt$data_source[5] <- "simulation"
SimKernelsInt$time_scale[5] <- "monsoonal"
SimKernelsInt$time_id[5] <- "NEM"
SimKernelsInt$k[5] <-  BestKNEM
SimKernelsInt$theta[5] <- BestThetaNEM
SimKernelsInt$num_matches[5] <-sum(biophys_par_dataNEM$Assignments[1:nrow(biophys_par_dataNEM$Assignments)-1,])
SimKernelsInt$mdd[5] <- MDDNEM
SimKernelsInt$med[5] <- MedNEM
SimKernelsInt$dist90[5] <- Dist90_NEM
SimKernelsInt$log_like[5] <- logLik(SimNEMFit)[1]

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

SimKernelsInt$iteration[6] <- i
SimKernelsInt$data_source[6] <- "simulation"
SimKernelsInt$time_scale[6] <- "monsoonal"
SimKernelsInt$time_id[6] <- "SWM"
SimKernelsInt$k[6] <-  BestKSWM
SimKernelsInt$theta[6] <- BestThetaSWM
SimKernelsInt$num_matches[6] <-sum(biophys_par_dataSWM$Assignments[1:nrow(biophys_par_dataSWM$Assignments)-1,])
SimKernelsInt$mdd[6] <- MDDSWM
SimKernelsInt$med[6] <- MedSWM
SimKernelsInt$dist90[6] <- Dist90_SWM
SimKernelsInt$log_like[6] <- logLik(SimSWMFit)[1]



#make a summary table for all time frames of genetic data
GenKernelsInt <- as.data.frame(matrix(NA, nrow = 6, ncol = 10))
setDT(GenKernelsInt)
setnames(GenKernelsInt, c("data_source", "time_scale", "time_id", "k", "theta", "num_matches", "mdd", "med", "dist90", "log_like"))
#GenKernelsInt <- data.table(data_source=time_scale=character(), time_id=character(), k=numeric(), theta=numeric(), mdd=numeric(), med=numeric(), dist90=numeric())


BestK2012 <- as.numeric(coef(Gen2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Gen2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[1] <- i
GenKernelsInt$data_source[1] <- "genetics"    
GenKernelsInt$time_scale[1] <- "annual"
GenKernelsInt$time_id[1] <- "2012"
GenKernelsInt$k[1] <-  BestK2012
GenKernelsInt$theta[1] <- BestTheta2012
GenKernelsInt$num_matches[1] <-sum(par_data2012$Assignments[1:nrow(par_data2012$Assignments)-1,])
GenKernelsInt$mdd[1] <- MDD2012
GenKernelsInt$med[1] <- Med2012
GenKernelsInt$dist90[1] <- Dist90_2012
GenKernelsInt$log_like[1] <- logLik(Gen2012Fit)[1]

BestK2013 <- as.numeric(coef(Gen2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Gen2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[2] <- i
GenKernelsInt$data_source[2] <- "genetics"    
GenKernelsInt$time_scale[2] <- "annual"
GenKernelsInt$time_id[2] <- "2013"
GenKernelsInt$k[2] <-  BestK2013
GenKernelsInt$theta[2] <- BestTheta2013
GenKernelsInt$num_matches[2] <-sum(par_data2013$Assignments[1:nrow(par_data2013$Assignments)-1,])
GenKernelsInt$mdd[2] <- MDD2013
GenKernelsInt$med[2] <- Med2013
GenKernelsInt$dist90[2] <- Dist90_2013
GenKernelsInt$log_like[2] <- logLik(Gen2013Fit)[1]

BestK2014 <- as.numeric(coef(Gen2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Gen2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[3] <- i
GenKernelsInt$data_source[3] <- "genetics"    
GenKernelsInt$time_scale[3] <- "annual"
GenKernelsInt$time_id[3] <- "2014"
GenKernelsInt$k[3] <-  BestK2014
GenKernelsInt$theta[3] <- BestTheta2014
GenKernelsInt$num_matches[3] <-sum(par_data2014$Assignments[1:nrow(par_data2014$Assignments)-1,])
GenKernelsInt$mdd[3] <- MDD2014
GenKernelsInt$med[3] <- Med2014
GenKernelsInt$dist90[3] <- Dist90_2014
GenKernelsInt$log_like[3] <- logLik(Gen2014Fit)[1]

BestK2012_4 <- as.numeric(coef(Gen2012_4Fit)[1])
BestTheta2012_4 <- as.numeric(coef(Gen2012_4Fit)[2])
MDD2012_4 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012_4, theta=BestTheta2012_4, , method = "pcubature")$integral)
k_eval <- BestK2012_4
theta_eval <- BestTheta2012_4
Med2012_4  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_2012_4 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[4] <- i
GenKernelsInt$data_source[4] <- "genetics"    
GenKernelsInt$time_scale[4] <- "total_years"
GenKernelsInt$time_id[4] <- "2012-4"
GenKernelsInt$k[4] <-  BestK2012_4
GenKernelsInt$theta[4] <- BestTheta2012_4
GenKernelsInt$num_matches[4] <-sum(par_data2012_4$Assignments[1:nrow(par_data2012_4$Assignments)-1,])
GenKernelsInt$mdd[4] <- MDD2012_4
GenKernelsInt$med[4] <- Med2012_4
GenKernelsInt$dist90[4] <- Dist90_2012_4
GenKernelsInt$log_like[4] <- logLik(Gen2012_4Fit)[1]

BestKNEM <- as.numeric(coef(GenNEMFit)[1])
BestThetaNEM <- as.numeric(coef(GenNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[5] <- i
GenKernelsInt$data_source[5] <- "genetics"    
GenKernelsInt$time_scale[5] <- "monsoonal"
GenKernelsInt$time_id[5] <- "NEM"
GenKernelsInt$k[5] <-  BestKNEM
GenKernelsInt$theta[5] <- BestThetaNEM
GenKernelsInt$num_matches[5] <-sum(par_dataNEM$Assignments[1:nrow(par_dataNEM$Assignments)-1,])
GenKernelsInt$mdd[5] <- MDDNEM
GenKernelsInt$med[5] <- MedNEM
GenKernelsInt$dist90[5] <- Dist90_NEM
GenKernelsInt$log_like[5] <- logLik(GenNEMFit)[1]

BestKSWM <- as.numeric(coef(GenSWMFit)[1])
BestThetaSWM <- as.numeric(coef(GenSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 4) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 4)

GenKernelsInt$iteration[6] <- i
GenKernelsInt$data_source[6] <- "genetics"    
GenKernelsInt$time_scale[6] <- "monsoonal"
GenKernelsInt$time_id[6] <- "SWM"
GenKernelsInt$k[6] <-  BestKSWM
GenKernelsInt$theta[6] <- BestThetaSWM
GenKernelsInt$num_matches[6] <-sum(par_dataSWM$Assignments[1:nrow(par_dataSWM$Assignments)-1,])
GenKernelsInt$mdd[6] <- MDDSWM
GenKernelsInt$med[6] <- MedSWM
GenKernelsInt$dist90[6] <- Dist90_SWM
GenKernelsInt$log_like[6] <- logLik(GenSWMFit)[1]

GenKernelsInt$iteration <- i
GenKernelsInt$data_source <- "genetics"

        BootKernelsInt <- rbind(SimKernelsInt, GenKernelsInt)
   
        BootKernels <- rbind(BootKernels, BootKernelsInt)
    
 #also bootstrap the biophysical likelihood
Data_par2012_bio2012 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2012[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012[1:nrow(genetic_parentage_matrices$GenMat2012)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2012,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012[nrow(genetic_parentage_matrices$GenMat2012),]))

Data_par2013_bio2013 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2013[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2013[1:nrow(genetic_parentage_matrices$GenMat2013)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2013,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2013[nrow(genetic_parentage_matrices$GenMat2013),]))

Data_par2014_bio2014 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$AnnualBiophysMatNorm2014[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2014[1:nrow(genetic_parentage_matrices$GenMat2014)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2014[nrow(genetic_parentage_matrices$GenMat2014),]))


#total compared to total
Data_par2012_4_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=genetic_parentage_matrices$GenMat2012_4[1:nrow(genetic_parentage_matrices$GenMat2012_4)-1,], 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMat2012_4[nrow(genetic_parentage_matrices$GenMat2012_4),]))

#for seasonal kernels

 Data_parNEM_bioNEM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormNEM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),]))

 Data_parSWM_bioSWM <- list(BioPhysMat=as.matrix(biophys_norm_matrices$MonsoonBiophysMatNormSWM[1:nrow(sampled_reefs_vec),]),
                    Assignments=as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,]), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))
#total to total 
 Data_parNEMSWM_bio2012_4 <- list(BioPhysMat=as.matrix(biophys_norm_matrices$FullBiophysMatNorm[1:nrow(sampled_reefs_vec),]),
                    Assignments=(as.matrix(genetic_parentage_matrices$GenMatNEM[1:nrow(genetic_parentage_matrices$GenMatNEM)-1,])+
                                  as.matrix(genetic_parentage_matrices$GenMatSWM[1:nrow(genetic_parentage_matrices$GenMatSWM)-1,])), 
                   pop_size_vec=as.matrix(SurveyData[,.(avg_num_females=mean(num_females, na.rm = TRUE)), by=site][order(site)][, .(avg_num_females)]), 
                   sampled_reefs_vec=as.matrix(SiteIndexBioPhys[site %in% SurveyData[, site], .(index)]),
                   prop_samp_vec=as.matrix(SurveyData[year == 2014,  .(prop_anem_samp)]), 
                   unassigned_vec=as.matrix(genetic_parentage_matrices$GenMatNEM[nrow(genetic_parentage_matrices$GenMatNEM),])+
                    as.matrix(genetic_parentage_matrices$GenMatSWM[nrow(genetic_parentage_matrices$GenMatSWM),]))

#calculate likelihood
LL_annual_annual <- LL_biophys(Data_par2012_bio2012)+LL_biophys(Data_par2013_bio2013)+LL_biophys(Data_par2014_bio2014)
LL_total_total <- LL_biophys(Data_par2012_4_bio2012_4)
LL_monsoonal_monsoonal <- LL_biophys(Data_parNEM_bioNEM)+LL_biophys(Data_parSWM_bioSWM)
LL_monsoonal_total <- LL_biophys(Data_parNEMSWM_bio2012_4)

#store results
LLInt <- data.table(comparison= c("gen_total_bio_total", "gen_annual_bio_annual", "gen_monsoonal_bio_monsoonal", "gen_monsoonal_bio_total"), 
           time_scale= c("total_years", "annual", "total_monsoons", "monsoonal"),
           log_like=c(LL_total_total, LL_annual_annual, LL_monsoonal_monsoonal, LL_monsoonal_total))

LLInt$iteration <- i
    
BootLikelihood <- rbind(BootLikelihood, LLInt)
    
    setTxtProgressBar(pb, i)
    }
close(pb)

#fwrite(BootKernels, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalKernels08DayPLD.csv")
#fwrite(BootLikelihood, "~/oceanography/script_output/KernelFits/summary_tables/BootstrappedBiophysicalLikelihoods08DayPLD.csv")

  |=                                                                     |   1%


In [8]:
head(BootLikelihood)

comparison,time_scale,log_like,iteration
gen_monsoonal_bio_total,monsoonal,-12147.39,1
gen_monsoonal_bio_monsoonal,total_monsoons,-22024.27,1
gen_total_bio_total,total_years,-25096.85,1
gen_annual_bio_annual,annual,-27249.29,1
gen_monsoonal_bio_total,monsoonal,-14297.00,2
gen_monsoonal_bio_monsoonal,total_monsoons,-26025.94,2
